# House Prices: Advanced Regression Techniques
## 予測モデリングの作成

<H2>1. 必要なライブラリを読み込む</H2>

In [23]:
% matplotlib inline

import copy
import os
import sys
import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

import pyodbc # データベースに接続するため

matplotlib.style.use('ggplot')

from __future__ import print_function

<H2>2. 定数定義</H2>

In [24]:
CONST_RANDOM_STATE = 1234
CONST_TEST_SIZE = 0.3
CONST_N_JOBS = -1
CONST_CUTOFF_R = 0.5
CONST_GRID_SEARCH_PARAMS = {'n_estimators': [200, 225, 250], 'max_depth': [15, 20, 25]}
#CONST_GRID_SEARCH_PARAMS = {'n_estimators': [215], 'max_depth': [15]}
CONST_FIRST_RMSLE = 1

<H2>3. グローバル変数定義</H2>

In [25]:
gRfParams = {'n_estimators' : 225, 'max_depth' : 20}
gDataSet = None
gDataSetTest = None
gColNames = None
gMissingCheck = None
gTargetCol = None
gExcludeCols = None
gFeatureCols = None
gRfr = None
gRmsle = CONST_FIRST_RMSLE
gNewXTrain1 = None
gNewXTrain2 = None
gNewXTrain = None
gNewXTest = None
gNewRealXTest = None

<H2>4. データの読み込み</H2>

In [26]:
def get_DataSet(sql) :
    # CSV ファイルからデータを読み込む
    # gDataSet = pd.read_csv('dataset/train.csv')
    
    # SQL Server からデータを読み込む
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=.\SQLEXPRESS;DATABASE=HousePrices;UID=sa;PWD=pass@word1')

    # Query database and load the returned results in pandas data frame
    dataSet = pd.read_sql(sql, conn)
    #dataSet = pd.read_sql('''select * from train''', conn)
    
    return dataSet

In [27]:
gDataSet = get_DataSet('select * from train')

In [28]:
gDataSetTest = get_DataSet('select * from test')

<h2>5. 特徴量の作成</h2>

<h3>ダミー変数を作る</h3>
<p> 一般的にはカテゴリ変数を機械学習のモデルに投入する際、0と1のダミー変数に変換する。</p>

In [29]:
# view のデータの種類を確認
# dataset['view'].unique()
# dataset = pd.get_dummies(data=dataset, columns=['view'])

<h3>カラム間の演算</h3>
<p>"LotFrontage/(LotArea/LotFrontage)"を新たな列として加える。土地の形を長方形と考えた場合において、間口と奥行きの比率を表す。</p>

In [30]:
# 学習データ
gDataSet['LotRatio'] = gDataSet['LotFrontage_Avg']/(gDataSet['LotArea']/gDataSet['LotFrontage_Avg'])
gDataSet.ix[0:9][['Id', 'LotArea', 'LotFrontage_Avg', 'LotRatio']]
# 検証データ
gDataSetTest['LotRatio'] = gDataSetTest['LotFrontage_Avg']/(gDataSetTest['LotArea']/gDataSetTest['LotFrontage_Avg'])
gDataSetTest.ix[0:9][['Id', 'LotArea', 'LotFrontage_Avg', 'LotRatio']]

C:\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
C:\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Id,LotArea,LotFrontage_Avg,LotRatio
0,1461,11622.0,80.0,0.550680
1,1462,14267.0,81.0,0.459872
2,1463,13830.0,74.0,0.395951
3,1464,9978.0,78.0,0.609741
4,1465,5005.0,43.0,0.369431
5,1466,10000.0,75.0,0.562500
6,1467,7980.0,65.0,0.529449
7,1468,8402.0,63.0,0.472388
8,1469,10176.0,85.0,0.710004
9,1470,8400.0,70.0,0.583333


<h2>6. ランダムフォレスト用いた予測モデルの構築</h2>

<p>ランダムフォレストを用いた機械学習アルゴリズムを試します</p>
<p>モデル評価はRMSLEとします</p>

In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error

<h3>ターゲット変数と特徴量を指定してsklearnに渡せるように準備する</h3>

In [32]:
gTargetCol = 'SalePrice'
gExcludeCols = ['SalePrice',
                'LotFrontage',
                'GarageYrBlt',
                'Alley',
                'BldgType',
                'BsmtCond',
                'BsmtExposure',
                'BsmtFinType1',
                'BsmtFinType2',
                'BsmtQual',
                'CentralAir',
                'Condition1',
                'Condition2',
                'Electrical',
                'ExterCond',
                'Exterior1st',
                'Exterior2nd',
                'ExterQual',
                'Fence',
                'FireplaceQu',
                'Foundation',
                'Functional',
                'GarageCond',
                'GarageFinish',
                'GarageQual',
                'GarageType',
                'Heating',
                'HeatingQC',
                'HouseStyle',
                'KitchenQual',
                'LandContour',
                'LandSlope',
                'LotConfig',
                'LotShape',
                'MasVnrType',
                'MiscFeature',
                'MSSubClass',
                'MSZoning',
                'Neighborhood',
                'OverallCond',
                'OverallQual',
                'PavedDrive',
                'PoolQC',
                'RoofMatl',
                'RoofStyle',
                'SaleCondition',
                'SaleType',
                'Street',
                'Utilities']
gFeatureCols = [col for col in gDataSet.columns if col not in gExcludeCols]

<H3>特徴量を割り算で作成</H3>

In [33]:
def get_NewAddedCols() :
    new_added_col = []
    for i in range(0, len(gFeatureCols)-1):
        for j in range(i+1, len(gFeatureCols)):
            first_col_name = gFeatureCols[i]
            second_col_name = gFeatureCols[j]
            r = spearmanr(gDataSet[first_col_name], gDataSet[second_col_name]).correlation
            if abs(r) > CONST_CUTOFF_R:
                new_colname = first_col_name + "_div_" + second_col_name
                gDataSet[new_colname] = gDataSet[first_col_name] / (gDataSet[second_col_name] + 0.01)
                gDataSetTest[new_colname] = gDataSetTest[first_col_name] / (gDataSetTest[second_col_name] + 0.01)
                new_added_col.append(new_colname)
    return new_added_col

In [34]:
gFeatureCols = gFeatureCols + get_NewAddedCols()
gFeatureCols

['Id',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'Alley_INT',
 'BldgType_INT',
 'BsmtCond_INT',
 'BsmtExposure_INT',
 'BsmtFinType1_INT',
 'BsmtFinType2_INT',
 'BsmtQual_INT',
 'CentralAir_INT',
 'Condition1_INT',
 'Condition2_INT',
 'Electrical_INT',
 'ExterCond_INT',
 'Exterior1st_INT',
 'Exterior2nd_INT',
 'ExterQual_INT',
 'Fence_INT',
 'FireplaceQu_INT',
 'Foundation_INT',
 'Functional_INT',
 'GarageCond_INT',
 'GarageFinish_INT',
 'GarageQual_INT',
 'GarageType_INT',
 'Heating_INT',
 'HeatingQC_INT',
 'HouseStyle_INT',
 'KitchenQual_INT',
 'LandContour_INT',
 'LandSlope_I

<H3>説明変数（特徴量）</H3>

In [35]:
gDataSet[gFeatureCols].head()

,Id,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,ExterQual_INT_div_HeatingQC_INT,ExterQual_INT_div_KitchenQual_INT,ExterQual_INT_div_OverallQual_INT,ExterQual_INT_div_GarageYrBlt_Oldest,Foundation_INT_div_GarageYrBlt_Oldest,GarageCond_INT_div_GarageQual_INT,KitchenQual_INT_div_OverallQual_INT,LandContour_INT_div_LandSlope_INT,OverallQual_INT_div_GarageYrBlt_Oldest,LotFrontage_Avg_div_LotRatio
0,1,8450.0,2003,2003,196,706,0,150,856,856,...,2.970297,0.996678,0.374532,0.001498,0.001498,0.998336,0.374532,3.960396,0.003994,127.450980
1,2,9600.0,1976,1976,0,978,0,284,1262,1262,...,3.960396,0.997506,0.570613,0.002024,0.001012,0.998336,0.570613,3.960396,0.003542,118.226601
2,3,11250.0,2001,2002,162,486,0,434,920,920,...,2.970297,0.996678,0.374532,0.001499,0.001499,0.998336,0.374532,3.960396,0.003998,161.511665
3,4,9550.0,1915,1970,0,216,0,540,756,961,...,1.328904,1.328904,0.499376,0.002002,0.000500,0.998336,0.374532,3.960396,0.004004,155.053443
4,5,14260.0,2000,2000,350,655,0,490,1145,1145,...,2.970297,0.996678,0.332963,0.001500,0.001500,0.998336,0.332963,3.960396,0.004500,166.399022


<H3>被説明変数（ターゲット変数）</H3>

In [36]:
gDataSet[gTargetCol].head()

0    208500.0
1    181500.0
2    223500.0
3    140000.0
4    250000.0
Name: SalePrice, dtype: float64

<H3>機械学習アルゴリズムに投入するための NumPy 配列</H3>

In [37]:
#被説明変数（ターゲット変数）のベクトル
gY = np.array(gDataSet[gTargetCol])
#NumPyの配列に直した特徴量の行列
gX = np.array(gDataSet[gFeatureCols])
gRealXTest = np.array(gDataSetTest[gFeatureCols])

In [38]:
# 学習データを70%(X_train, y_train)、テストデータを30%(X_test, y_test)に分割
gX_train, gX_test, gY_train, gY_test = train_test_split(gX, gY, test_size=CONST_TEST_SIZE, random_state=CONST_RANDOM_STATE)

In [39]:
# さらに分割した学習データを70%(X_train1, y_train1)と30%(X_train2, y_train2)に分割
gX_train1, gX_train2, gY_train1, gY_train2 = train_test_split(gX_train, gY_train, test_size=CONST_TEST_SIZE, random_state=CONST_RANDOM_STATE)

<H3>ランダムフォレストの実行</H3>

<h5>ランダムフォレストの適合</h4>

In [40]:
def GetNewFeatures(mask, feature_names) :
    new_features = [] 
    for bool, feature in zip(mask, feature_names) :
        if bool :
            new_features.append(feature)
    return new_features       

In [41]:
def GetRmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

In [42]:
def fit_RandomForestRegressorTrain(x_train1, y_train1, x_train2, y_train2, x_train, y_train, x_test, y_test, x_real_test, features_name, params, th) :
    lFeatureCount = len(features_name)
    rmsle = 0
    
    ### 2. モデルベース特徴量選択 ###
    model = RandomForestRegressor(random_state=CONST_RANDOM_STATE, n_estimators=params['n_estimators'], max_depth=params['max_depth'], n_jobs=CONST_N_JOBS)
    select = SelectFromModel(estimator=model, threshold=th)
    select.fit(x_train1, y_train1)
    new_x_train1 = select.transform(x_train1)
    new_x_train2 = select.transform(x_train2)
    new_x_train = select.transform(x_train)
    new_x_test = select.transform(x_test)
    new_x_real_test = select.transform(x_real_test)
    new_features_name = GetNewFeatures(select.get_support(), features_name)
    
    ### 4. パラメーターチューニング ###
    if len(new_features_name) < lFeatureCount :
        gscv = GridSearchCV(model, param_grid=CONST_GRID_SEARCH_PARAMS, scoring='neg_mean_squared_log_error')
        gscv.fit(new_x_train1, y_train1)    
        params['n_estimators'] = gscv.best_params_['n_estimators']
        params['max_depth'] = gscv.best_params_['max_depth']
    
    ### 5. [Test] モデルの適合04
    if len(new_features_name) < lFeatureCount :
        model = RandomForestRegressor(random_state=CONST_RANDOM_STATE, n_estimators=params['n_estimators'], max_depth=params['max_depth'], n_jobs=CONST_N_JOBS)
        model.fit(new_x_train, y_train)  
        y_pred = model.predict(new_x_test)
        rmsle = GetRmsle(y_test, y_pred)
   
    return new_x_train1, new_x_train2, new_x_train, new_x_test, new_x_real_test, new_features_name, rmsle, params, model, select

<h5>クロスバリデーションによるパラメーターのチューニング</h5>

どのようなハイパーパラメーターがあるかは公式ドキュメントでチェックする
<br>
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

scikit-learnのバージョンを確認する

In [43]:
import sklearn 
sklearn.__version__

'0.19.0'

ハイパーパラメーターのチューニングをGridSearchで行う

In [44]:
# 特徴量データの初期値
gNewXTrain1 = gX_train1
gNewXTrain2 = gX_train2
gNewXTrain = gX_train
gNewXTest = gX_test
gNewRealXTest = gRealXTest
gNewFeatureNames = gFeatureCols

# 変数の初期化
lNewRmsle = 0
lLoop = 0
lThreshold = None

    
# 結果出力用のファイル
lFile = open('D:\\DataMix\\08.170909\\House Prices Advanced Regression Techniques\\result\\Result_15-SelectFromModel_'
             + datetime.datetime.now().strftime('%Y%m%d-%H%M%S') + '.csv', 'a')

while True:
    
    # ループ回数
    lLoop = lLoop + 1
    
    # 閾値
    lThreshold = lLoop * 0.0001
    
    # パラメータの記憶
    lFeatureCount = len(gNewFeatureNames)

    # DEBUG
    print("【{0}回目-START】閾値:{1}, RMSLE:{2}, 特徴量数:{3}, n_estimators:{4}, max_depth:{5}".format(
                            lLoop, round(lThreshold,4), round(lNewRmsle, 5), len(gNewFeatureNames), gRfParams["n_estimators"], gRfParams["max_depth"]))
    
    # 学習と検証
    gNewXTrain1, gNewXTrain2, gNewXTrain, gNewXTest, gNewRealXTest, gNewFeatureNames, lNewRmsle, gRfParams, gModel, gSelectFromModel = \
        fit_RandomForestRegressorTrain(
            x_train1 = gNewXTrain1,
            y_train1 = gY_train1,
            x_train2 = gNewXTrain2,
            y_train2 = gY_train2,
            x_train = gNewXTrain,
            y_train = gY_train,
            x_test = gNewXTest,
            y_test = gY_test,
            x_real_test = gNewRealXTest,
            features_name = gNewFeatureNames, 
            params = gRfParams,
            th = lThreshold)

    # 特徴量に変化があったかどうか
    if len(gNewFeatureNames) < lFeatureCount :
 
        # 結果出力
        lOutput = "\"" + str(lLoop) + "\",\"" + str(round(lThreshold,4)) + "\",\"" + str(round(lNewRmsle, 5)) + "\",\"" + str(len(gNewFeatureNames)) + "\",\"" + str(gRfParams["n_estimators"]) + "\",\"" + str(gRfParams["max_depth"])
        lOutput = lOutput + "\",[\""
        for x in gNewFeatureNames :
            lOutput = lOutput + x + "\",\"" 
        lOutput = lOutput[:-2] + "]\n"
        lFile.write(lOutput)
        
        # 予測値の導出
        y_pred_on_test = gModel.predict(gNewRealXTest)
        
        # 予測値の出力
        f = open('D:\\DataMix\\08.170909\\House Prices Advanced Regression Techniques\\dataset\\17\\y_pred_' + str(len(gNewFeatureNames)) + '.csv', 'w')
        for x in y_pred_on_test:
            f.write(str(x) + "\n")
        f.close()
    
    # 終了判定
    if lThreshold > 0.9 :
        lFile.close()   
        break

    # DEBUG
    print("【{0}回目-END  】閾値:{1}, RMSLE:{2}, 特徴量数:{3}, n_estimators:{4}, max_depth:{5}".format(
                           lLoop, round(lThreshold,4), round(lNewRmsle, 5), len(gNewFeatureNames), gRfParams["n_estimators"], gRfParams["max_depth"]))
       
# 最終結果
print('完了')

【1回目-START】閾値:0.0001, RMSLE:0, 特徴量数:146, n_estimators:225, max_depth:20
【1回目-END  】閾値:0.0001, RMSLE:0.1498, 特徴量数:130, n_estimators:200, max_depth:20
【2回目-START】閾値:0.0002, RMSLE:0.1498, 特徴量数:130, n_estimators:200, max_depth:20
【2回目-END  】閾値:0.0002, RMSLE:0.14958, 特徴量数:117, n_estimators:200, max_depth:25
【3回目-START】閾値:0.0003, RMSLE:0.14958, 特徴量数:117, n_estimators:200, max_depth:25
【3回目-END  】閾値:0.0003, RMSLE:0.14835, 特徴量数:108, n_estimators:200, max_depth:15
【4回目-START】閾値:0.0004, RMSLE:0.14835, 特徴量数:108, n_estimators:200, max_depth:15
【4回目-END  】閾値:0.0004, RMSLE:0.14934, 特徴量数:99, n_estimators:200, max_depth:15
【5回目-START】閾値:0.0005, RMSLE:0.14934, 特徴量数:99, n_estimators:200, max_depth:15
【5回目-END  】閾値:0.0005, RMSLE:0.14911, 特徴量数:97, n_estimators:200, max_depth:15
【6回目-START】閾値:0.0006, RMSLE:0.14911, 特徴量数:97, n_estimators:200, max_depth:15
【6回目-END  】閾値:0.0006, RMSLE:0.14942, 特徴量数:89, n_estimators:200, max_depth:25
【7回目-START】閾値:0.0007, RMSLE:0.14942, 特徴量数:89, n_estimators:200, max_depth:25


【55回目-END  】閾値:0.0055, RMSLE:0, 特徴量数:37, n_estimators:200, max_depth:25
【56回目-START】閾値:0.0056, RMSLE:0, 特徴量数:37, n_estimators:200, max_depth:25
【56回目-END  】閾値:0.0056, RMSLE:0, 特徴量数:37, n_estimators:200, max_depth:25
【57回目-START】閾値:0.0057, RMSLE:0, 特徴量数:37, n_estimators:200, max_depth:25
【57回目-END  】閾値:0.0057, RMSLE:0.15483, 特徴量数:36, n_estimators:200, max_depth:20
【58回目-START】閾値:0.0058, RMSLE:0.15483, 特徴量数:36, n_estimators:200, max_depth:20
【58回目-END  】閾値:0.0058, RMSLE:0, 特徴量数:36, n_estimators:200, max_depth:20
【59回目-START】閾値:0.0059, RMSLE:0, 特徴量数:36, n_estimators:200, max_depth:20
【59回目-END  】閾値:0.0059, RMSLE:0, 特徴量数:36, n_estimators:200, max_depth:20
【60回目-START】閾値:0.006, RMSLE:0, 特徴量数:36, n_estimators:200, max_depth:20
【60回目-END  】閾値:0.006, RMSLE:0.15595, 特徴量数:35, n_estimators:200, max_depth:15
【61回目-START】閾値:0.0061, RMSLE:0.15595, 特徴量数:35, n_estimators:200, max_depth:15
【61回目-END  】閾値:0.0061, RMSLE:0, 特徴量数:35, n_estimators:200, max_depth:15
【62回目-START】閾値:0.0062, RMSLE:0, 特徴量数:35, n

【111回目-END  】閾値:0.0111, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【112回目-START】閾値:0.0112, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【112回目-END  】閾値:0.0112, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【113回目-START】閾値:0.0113, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【113回目-END  】閾値:0.0113, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【114回目-START】閾値:0.0114, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【114回目-END  】閾値:0.0114, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【115回目-START】閾値:0.0115, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【115回目-END  】閾値:0.0115, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【116回目-START】閾値:0.0116, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【116回目-END  】閾値:0.0116, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【117回目-START】閾値:0.0117, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【117回目-END  】閾値:0.0117, RMSLE:0, 特徴量数:22, n_estimators:200, max_depth:20
【118回目-START】閾値:0.0118, RMSLE:0, 特徴量数:22, n_estimat

【167回目-END  】閾値:0.0167, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【168回目-START】閾値:0.0168, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【168回目-END  】閾値:0.0168, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【169回目-START】閾値:0.0169, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【169回目-END  】閾値:0.0169, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【170回目-START】閾値:0.017, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【170回目-END  】閾値:0.017, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【171回目-START】閾値:0.0171, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【171回目-END  】閾値:0.0171, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【172回目-START】閾値:0.0172, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【172回目-END  】閾値:0.0172, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【173回目-START】閾値:0.0173, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【173回目-END  】閾値:0.0173, RMSLE:0, 特徴量数:18, n_estimators:200, max_depth:20
【174回目-START】閾値:0.0174, RMSLE:0, 特徴量数:18, n_estimator

【223回目-END  】閾値:0.0223, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【224回目-START】閾値:0.0224, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【224回目-END  】閾値:0.0224, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【225回目-START】閾値:0.0225, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【225回目-END  】閾値:0.0225, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【226回目-START】閾値:0.0226, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【226回目-END  】閾値:0.0226, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【227回目-START】閾値:0.0227, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【227回目-END  】閾値:0.0227, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【228回目-START】閾値:0.0228, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【228回目-END  】閾値:0.0228, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【229回目-START】閾値:0.0229, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【229回目-END  】閾値:0.0229, RMSLE:0, 特徴量数:15, n_estimators:225, max_depth:15
【230回目-START】閾値:0.023, RMSLE:0, 特徴量数:15, n_estimato

...

【1305回目-END  】閾値:0.1305, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1306回目-START】閾値:0.1306, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1306回目-END  】閾値:0.1306, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1307回目-START】閾値:0.1307, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1307回目-END  】閾値:0.1307, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1308回目-START】閾値:0.1308, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1308回目-END  】閾値:0.1308, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1309回目-START】閾値:0.1309, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1309回目-END  】閾値:0.1309, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1310回目-START】閾値:0.131, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1310回目-END  】閾値:0.131, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1311回目-START】閾値:0.1311, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1311回目-END  】閾値:0.1311, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1312回目-START】閾値:0.1312, RMSLE:0, 特徴量数:4, n_estimator

【1362回目-END  】閾値:0.1362, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1363回目-START】閾値:0.1363, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1363回目-END  】閾値:0.1363, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1364回目-START】閾値:0.1364, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1364回目-END  】閾値:0.1364, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1365回目-START】閾値:0.1365, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1365回目-END  】閾値:0.1365, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1366回目-START】閾値:0.1366, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1366回目-END  】閾値:0.1366, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1367回目-START】閾値:0.1367, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1367回目-END  】閾値:0.1367, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1368回目-START】閾値:0.1368, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1368回目-END  】閾値:0.1368, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1369回目-START】閾値:0.1369, RMSLE:0, 特徴量数:4, n_estimat

【1419回目-END  】閾値:0.1419, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1420回目-START】閾値:0.142, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1420回目-END  】閾値:0.142, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1421回目-START】閾値:0.1421, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1421回目-END  】閾値:0.1421, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1422回目-START】閾値:0.1422, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1422回目-END  】閾値:0.1422, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1423回目-START】閾値:0.1423, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1423回目-END  】閾値:0.1423, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1424回目-START】閾値:0.1424, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1424回目-END  】閾値:0.1424, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1425回目-START】閾値:0.1425, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1425回目-END  】閾値:0.1425, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1426回目-START】閾値:0.1426, RMSLE:0, 特徴量数:4, n_estimator

【1476回目-END  】閾値:0.1476, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1477回目-START】閾値:0.1477, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1477回目-END  】閾値:0.1477, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1478回目-START】閾値:0.1478, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1478回目-END  】閾値:0.1478, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1479回目-START】閾値:0.1479, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1479回目-END  】閾値:0.1479, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1480回目-START】閾値:0.148, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1480回目-END  】閾値:0.148, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1481回目-START】閾値:0.1481, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1481回目-END  】閾値:0.1481, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1482回目-START】閾値:0.1482, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1482回目-END  】閾値:0.1482, RMSLE:0, 特徴量数:4, n_estimators:225, max_depth:20
【1483回目-START】閾値:0.1483, RMSLE:0, 特徴量数:4, n_estimator

KeyboardInterrupt: 